In [1]:
import torch
import torch.nn as nn
import tiktoken

In [2]:
from gpt_model import *

In [27]:
# to make training faster on a laptop, change context_length as shown:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 256,  # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
model = GPTModel(GPT_CONFIG_124M)

In [5]:
start_context = "Every effort moves you"

In [8]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you inventory Doe rule ranger headquartered scanners Koch 44 jailed Tac


In [10]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you forward",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [11]:
with torch.no_grad():
    logits = model(inputs)

probs = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probs.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [12]:
probs

tensor([[[1.8487e-05, 8.3284e-06, 2.8235e-05,  ..., 3.8520e-06,
          1.4658e-05, 9.3012e-06],
         [1.3775e-05, 2.3699e-05, 7.9553e-06,  ..., 1.1093e-05,
          6.1530e-06, 3.1080e-05],
         [1.7811e-05, 1.4435e-05, 1.1795e-05,  ..., 1.4894e-05,
          2.4188e-05, 3.3994e-05]],

        [[1.8961e-05, 1.3415e-05, 2.1579e-05,  ..., 9.6470e-06,
          8.7392e-06, 5.5270e-06],
         [1.6604e-05, 1.5070e-05, 1.3294e-05,  ..., 3.0797e-05,
          9.1548e-06, 1.7088e-05],
         [3.3181e-05, 1.4161e-05, 2.0338e-05,  ..., 5.0536e-05,
          1.5840e-05, 2.2130e-05]]])

In [13]:
# predicted tokens:
token_ids = torch.argmax(probs, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[ 7663],
         [19259],
         [36848]],

        [[44358],
         [17083],
         [19333]]])


In [14]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Fieldvance Supported


In [15]:
print(f"Targets batch 2: {token_ids_to_text(targets[1], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")

Targets batch 2:  really like chocolate
Outputs batch 2:  honorary Together Archer


In [16]:
targets

tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])

In [17]:
text_idx = 0
target_probs_1 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_1)

text_idx = 1
target_probs_2 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probs_2)

Text 1: tensor([2.4905e-05, 1.4786e-05, 1.7707e-05])
Text 2: tensor([1.4995e-05, 1.2197e-05, 1.1763e-05])


In [20]:
torch.set_printoptions( sci_mode=False )

In [21]:
# Compute logarithm of all token probabilities
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
print(log_probs)

tensor([-10.6005, -11.1219, -10.9416, -11.1078, -11.3143, -11.3506])


In [22]:
avg_log_probs = torch.mean(log_probs)
print(avg_log_probs)
neg_avg_log_probs = avg_log_probs * -1
print(neg_avg_log_probs)

tensor(-11.0728)
tensor(11.0728)


In [23]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


In [24]:
print(logits_flat)
print(targets_flat)

tensor([[ 0.0913, -0.7061,  0.5148,  ..., -1.4772, -0.1408, -0.5956],
        [-0.2021,  0.3405, -0.7511,  ..., -0.4186, -1.0080,  0.6117],
        [ 0.0524, -0.1578, -0.3598,  ..., -0.1265,  0.3584,  0.6987],
        [ 0.1188, -0.2271,  0.2482,  ..., -0.5569, -0.6557, -1.1139],
        [-0.0157, -0.1126, -0.2380,  ...,  0.6021, -0.6110,  0.0131],
        [ 0.6756, -0.1759,  0.1861,  ...,  1.0963, -0.0639,  0.2705]])
tensor([ 3626,  6100,   345,  1107,   588, 11311])


In [25]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(11.0728)


In [26]:
# a more interpretable version of cross entropy --
# this is basically the number of tokens the model considers for predicted output
perplexity = torch.exp(loss)
print(perplexity)

tensor(64392.8398)


In [31]:
# use the short story from before for training:
with open( "humphreys.txt", "r" ) as f:
    text_data = f.read()
print(text_data[:100])

ABOUT fifteen years ago, on a date late in August or early in September, a train drew up at Wilsthor


In [37]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [38]:
# separate text into training and validation sets:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [39]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 12800
Validation tokens: 1280
All tokens: 14080


In [40]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [41]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    # Use PyTorch 2.9 or newer for stable mps results
    major, minor = map(int, torch.__version__.split(".")[:2])
    if (major, minor) >= (2, 9):
        device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Using cpu device.
Training loss: 10.970036811828614
Validation loss: 10.978055000305176
